# Prereading

Familiarize yourself with the following.


## Antithetic Sampling

Suppose estimator $\hat \theta_1$ and $\hat \theta_2$ have $E[\hat \theta_1] = E[\hat \theta_2] = \theta$ and correlation $\rho = \frac{Cov(\hat \theta_1, \hat \theta_2)}{\sigma_{\hat \theta_1}\sigma_{\hat \theta_2}}$. 

If (for simplicity of demonstration) $\sigma_{\hat \theta_1}^2=\sigma_{\hat \theta_2}^2=\frac{\sigma^2}{n}$ then 

$$
\begin{align*}
\text{Var}\left(\frac{\hat \theta_1+\hat \theta_2}{2}\right) = {} & \frac{\text{Var}(\hat \theta_1)+\text{Var}(\hat \theta_2)}{4} + \frac{2\text{Cov}(\hat \theta_1, \hat \theta_2)}{4} \\
= {} & \frac{\sigma^2}{2n} + \frac{\frac{\sigma}{\sqrt{n}}\frac{\sigma}{\sqrt{n}}\rho}{2} = \frac{(1+\rho)\sigma^2}{2n}
\end{align*}$$

If $\rho<0$ then 
$\text{Var}\left(\frac{\hat \theta_1+\hat \theta_2}{2}\right) = \frac{(1+\rho)\sigma^2}{2n}$
is less than $\text{Var}(\hat \theta_1)$ and $\text{Var}(\hat \theta_2)$.



Achieving $\rho<0$ (and potentially a substantially lower estimation variance) can in fact be easy... 

For example, if $\hat \theta_k = g(\mathbf{x})$ is a function of $x_i$ such that $\rho_{\hat \theta_k,x_i} > 0$

- $\rho_{\hat \theta_1,\hat \theta_2} < 0 \text{ if } x_{i}' = -x_{i} \text{ for } x_{i}\sim N(\mu, \sigma^2_x)$
- $\rho_{\hat \theta_1,\hat \theta_2} < 0 \text{ if } x_{i}' = -x_{i} \text{ for } x_{i}\sim U(0,1)$
- $\rho_{\hat \theta_1,\hat \theta_2} < 0 \text{ if } x_{i}' = F_X^{-1}(1-F_X(x_{i})) \text{ for } x_{i}\sim f(X)$


## Continuing from the HW...

$$\Large 
\begin{align*}
p(x) &={} \textrm{truncexpon}(5,1)\\
g(x) &={} 5e^{-5x}\times \int_0^1 5e^{-5x}dx\\
\theta &={} \int_0^1 h(x)g(x)p(x)dx = {\left\{\begin{array}{rl}\int_0^1 g(x)\frac{p(x)}{q(x)}q(x)dx {}:& \textrm{if } h(x)=1\\\int_0^1 x g(x)\frac{p(x)}{q(x)}q(x)dx {}:& \textrm{if } h(x)=x\end{array}\right.}
\end{align*}$$


In [ ]:
def experiment(x=np.linspace(0,1,1001), n=100,
               h = lambda x: 1, 
               g = lambda x: stats.expon(scale=1/5).pdf(x)*stats.expon(scale=1/5).cdf(1),
               p = lambda x: stats.truncexpon(scale=1/5, b=5).pdf(x),
               q = stats.beta(1,1), 
               plotting=True, antithetic_sampling=False, control_variates=False):

    x_ = q.rvs(n)
    w_ = p(x_)/q.pdf(x_)
    print()
    if antithetic_sampling:
        x__ = q.ppf(1-q.cdf(x_))    
        w__ = p(x__)/q.pdf(x__)
    
    if plotting:
        if antithetic_sampling:
            fig,ax = plt.subplots(3,3, figsize=(10,9))
        else:
            fig,ax = plt.subplots(2,3, figsize=(10,6))
        theta = integrate.quad(lambda x: h(x)*g(x)*p(x), 0, 1)[0]
        evals_ = h(x_)*g(x_)*w_
        ax[1,0].hist(evals_, label="h*g*p/q ~q\n std = %.3f"%evals_.std(), alpha=0.5)
        #ax[0,0].plot(x, h(x)*g(x), label="h*g")
        ax[0,0].plot(x, h(x)*g(x)*p(x), label="h*g*p theta="+str("%.3f"%theta))
        ax[0,0].plot(x, h(x)*g(x)*p(x)/q.pdf(x), label="h*g*p/q")
        ax[0,0].plot(x, q.pdf(x), label="q")
        ax[0,0].set_ylim((0, evals_.max()))
        ax[0,0].legend()
        ax[0,1].plot(x, p(x), label="p")        
        ax[0,1].plot(x, q.pdf(x), label="q")
        ax[0,1].legend()
        if control_variates:
            C_ = (evals_-evals_.mean()).T.dot(w_-1)/(n-1)  # np.cov(evals_,w_)[0,1]#/n
            V_ = ((w_-1)**2).mean()  # np.var(w_)#/n
            est_cv_ = evals_.mean() - C_/V_*(w_.mean() - 1)  # 𝜆 = E[w_] = 1        
        if antithetic_sampling:
            evals__ = h(x__)*g(x__)*w__
            ax[1,0].hist(evals__, label="h*g*p/q ~q-anti\n std = %.3f"%evals__.std(), alpha=0.5)
            anti_cor = np.corrcoef(evals_, evals__)[0,1]
            anti_SE = (((1+anti_cor)*(evals_.var()+evals__.var()))/(4*n))**0.5
            ax[2,0].plot(evals_, evals__, '.', 
                         label="h*g*p/q   \n ~q vs ~q-anti\n cor = %.3f\n SE = %.3f"%(anti_cor,anti_SE),)
            if control_variates:
                C__ = (evals__-evals__.mean()).T.dot(w__-1)/(n-1)  # np.cov(evals__, w__)[0,1]#/n
                V__ = ((w__-1)**2).mean()  # np.var(w__, ddof=1)#/n
                est_cv__ = evals__.mean() - C__/V__*(w__.mean() - 1)  # 𝜆 = E[w_] = 1
                est_cv = (est_cv_+est_cv__)/2
                ax[2,1].vlines([est_cv_, est_cv], evals__.min(), evals__.max(), linestyle=":", 
                               color=['k','r'], label='control variates ~q')
                ax[2,1].hlines([est_cv__, est_cv], evals_.min(), evals_.max(), linestyle=":", 
                               color=['k','r'], label='control variates ~q-anti')
                ax[2,1].text(est_cv_, evals__.min(), "%.3f\n SE=%.3f"%(est_cv_, ((evals_.var()-C_**2/V_)/n)**0.5))
                ax[2,1].text(evals_.min(), est_cv__, "%.3f\n SE=%.3f"%(est_cv__, ((evals__.var()-C__**2/V__)/n)**0.5))
                ax[2,1].text(est_cv, est_cv, "%.3f"%est_cv)   
                ax[2,1].set_xlim([-0.1*theta,2.1*theta])
                ax[2,1].set_ylim([-0.1*theta,2.1*theta])
                ax[2,1].plot(theta, theta, 'go', label="theta = %.3f"%theta)
                ax[2,1].legend()
                ax[2,2].bar(height=[-C_/V_*(w_.mean()-1), -C__/V__*(w__.mean()-1)], 
                            x=[0,1], label="cov(xw*,w*)/n\n = %.3f ~q\n cov(xw*,w*)/n\n = %.3f ~q-anti"%(C_/n,C__/n))
                ax[2,2].set_xticks([0,1],["C/V(w-bar - 1)\n ~q", "C/V(w-bar - 1)\n ~q-anti"])
                ax[2,2].plot([-0.5,1.5],[0,0],'k')
                ax[2,2].legend()

            est_ = evals_.mean()
            est__ = evals__.mean()
            est = (est_+est__)/2
            ax[2,0].vlines([est_, est], evals__.min(), evals__.max(), color=['k','r'])
            ax[2,0].hlines([est__, est], evals_.min(), evals_.max(), color=['k','r'])
            ax[2,0].text(est_, evals__.min(), "%.3f\n SE %.3f"%(est_, evals_.std()/n**0.5))
            ax[2,0].text(evals_.min(), est__, "%.3f\n SE %.3f"%(est__, evals__.std()/n**0.5))
            ax[2,0].text(est, est, "%.3f"%est)
            ax[2,0].set_xlim([-0.1*theta,2.1*theta])
            ax[2,0].set_ylim([-0.1*theta,2.1*theta])
            ax[2,0].plot(theta, theta, 'go', label="theta = %.3f"%theta)
            ax[2,0].legend(loc = 'upper right')
            
        ax[0,2].plot(x, p(x)/q.pdf(x), label="p/q")
        ax[0,2].plot(x_, w_, '.',label="p/q ~q")
        ax[0,2].set_ylim((w_.min()-0.1*(w_.max()-w_.min()),
                          w_.max()+0.1*(w_.max()-w_.min())))
        ax[0,2].legend()
        ax[1,0].legend()        
        ax[1,1].plot(w_, h(x_)*g(x_), '.', label="h*g ~q\n versus\n p/q ~q")
        ax[1,1].legend()
        n_eff = 1/((w_/w_.sum())**2).sum()
        ax[1,2].hist(w_, label="p/q ~q\n std = %.3f\n n_eff = %.3f"%(w_.std(),n_eff))
        ax[1,2].legend()
        plt.tight_layout()

    if antithetic_sampling:
        return (h(x_)*g(x_)*w_).mean(),\
               (h(x__)*g(x__)*w__).mean(), 
    return (h(x_)*g(x_)*w_).mean()

In [ ]:
# try it out...


## Bagging 

**Antithetic sampling** is somewhat similar to **Bagging** in ***Random Forests*** where for $k=1,\cdots, K$ tree-based precictions $\hat t_k$ have in expectation the same common variance $\sigma^2_{t}$ and shared correlation $\rho = \frac{Cov(t_k,t_{k'})}{\sigma^2_{t}} > 0$. Then a generalization of
$\text{Var}\left(\frac{\hat t_1+\hat t_2}{2}\right) = \frac{\text{Var}(\hat t_1)+\text{Var}(\hat t_2)}{4} + \frac{2\text{Cov}(\hat t_1+\hat t_2)}{4}$ to $K$ rather than $2$

$$
Var(\bar t) = \underbrace{\frac{1}{K^2} \sum_{k=1}^K \sigma^2_t}_{\textrm{sum of variances}} + \underbrace{\frac{2}{K^2} \sum_{k,k'} \sigma_t^2 \rho}_{\textrm{sum of covariances}} = \frac{\sigma^2}{K} + \frac{\frac{2}{2}(K^2-K)\sigma_t^2\rho}{K^2}
 = \rho \sigma^2 + \frac{(1-\rho)\sigma^2}{K}
$$

which means there are benefits to be had from averaging even positively correlated estimators.

# Lecture

First hour of class

## Control Variates

$
\large
\begin{align*}
[\hat \theta + \beta (\hat \lambda - \lambda)]\\
Var[\hat \theta + \beta (\hat \lambda - \lambda)] &={}
Var[\hat \theta] + \beta^2Var[\hat \lambda] + 2\beta Cov(\hat \theta, \hat \lambda)\\
& \frac{d}{d\beta}{}
Var[\hat \theta] + \beta^2 Var[\hat \lambda] + 2 \beta Cov(\hat \theta, \hat \lambda) \;\; = \;\; 0 \\
&={} 2\beta Var[\hat \lambda] + 2 Cov(\hat \theta, \hat \lambda) \quad \Longrightarrow \quad \beta \quad = \quad - \frac{Cov(\hat \theta, \hat \lambda)}{Var[\hat \lambda]} \\
\hat \theta - \frac{Cov(\hat \theta, \hat \lambda)}{Var[\hat \lambda]} (\hat \lambda - \lambda)\\
Var[\hat \theta + \beta (\hat \lambda - \lambda)] &={}
Var[\hat \theta] - \frac{Cov(\hat \theta, \hat \lambda)}{Var[\hat \lambda]}^2 < Var[\hat \theta] \\\\\textbf{Importance Sampling}\\
Cov(\hat \theta, \hat \lambda) &={} Cov\left(\frac{1}{n}\sum_i x_iw^*_i, \frac{1}{n}\sum_i w^*_i\right) \\
&={} \sum_i Cov\left( \frac{1}{n} x_iw^*_i, \frac{1}{n} w^*_i\right) = \frac{1}{n} Cov\left(x_iw^*_i, w^*_i\right) \\
\end{align*}
$


# Class Coding Challenge

This is
- in person in approximately the final two hours of class
- open notes and open internet (including AI chat bot support tools) **but you may not collaborate with other humans (like classmates, etc.)**

This will 

- utilize the coding skills practiced in the prelecture homework
- extend the considerations practiced in the prelecture homework
- address topics from the prelecture reading and lectures itself

and is cumulative in nature relative to all content of the course.